In [1]:
import sys
sys.path.append("../../")

In [4]:
from didgelab.calc.conv import note_to_freq
from didgelab.evo.loss import LossFunction

from didgelab.evo.evolution import MultiEvolution
from didgelab.initializer import init_console
from didgelab.app import get_config, get_app

from didgelab.calc.sim.sim import compute_impedance_iteratively, get_notes, compute_impedance, create_segments
from didgelab.calc.geo import Geo

import math
import numpy as np

# a loss that deviates 
def single_note_loss(note, peaks, i_note=0, filter_rel_imp=0.1):
    print(peaks)
    peaks=peaks[peaks.rel_imp>filter_rel_imp]
    if len(peaks)<=i_note:
        return 1000000
    f_target=note_to_freq(note)
    f_fundamental=peaks.iloc[i_note]["freq"]
    return np.sqrt(abs(math.log(f_target, 2)-math.log(f_fundamental, 2)))

# add loss if the didge gets smaller
def diameter_loss(geo):

    if type(geo)==Geo:
        shape=geo.geo
    elif type(geo) == list:
        shape=geo
    else:
        raise Exception("unknown type " + str(type(geo)))

    loss=0
    for i in range(1, len(shape)):
        delta_y=shape[i-1][1]-shape[i][1]
        if delta_y < 0:
            loss+=-1*delta_y

    loss*=0.005
    return loss


class MbeyaLoss(LossFunction):

    # fundamental: note number of the fundamental
    # add_octave: the first toot is one octave above the fundamental
    # scale: define the scale of the toots of the didgeridoo as semitones relative from the fundamental
    # target_peaks: define the target peaks as list of math.log(frequency, 2). overrides scale 
    # n_notes: set > 0 to determine the number of impedance peaks (above fundamental and add_octave)
    # weights: override the default weights
    # {
    #     "tuning_loss": 8,
    #     "volume_loss": 0.5,
    #     "octave_loss": 4,
    #     "n_note_loss": 5,
    #     "diameter_loss": 0.1,
    #     "fundamental_loss": 8,
    # }
    def __init__(self, fundamental=-31, add_octave=True, n_notes=-1, scale=[0,2,3,5,7,9,10], target_peaks=None, weights={}):
        LossFunction.__init__(self)

        self.weights={
            "tuning_loss": 8,
            "volume_loss": 0.5,
            "octave_loss": 4,
            "n_note_loss": 5,
            "diameter_loss": 0.1,
            "fundamental_loss": 8,
        }
        for key, value in weights.items():
            if key not in self.weights:
                raise Exception(f"Unknown weight {key}")
            self.weights[key]=value


        self.scale=scale
        self.fundamental=fundamental
        self.add_octave=add_octave
        self.n_notes=n_notes

        if target_peaks is not None:
            self.target_peaks=target_peaks
        else:
            self.scale_note_numbers=[]
            for i in range(len(self.scale)):
                self.scale_note_numbers.append(self.scale[i]+self.fundamental)

            n_octaves=10
            self.target_peaks=[]
            for note_number in self.scale_note_numbers:
                for i in range(0, n_octaves):
                    transposed_note=note_number+12*i
                    freq=note_to_freq(transposed_note)
                    freq=math.log(freq, 2)
                    self.target_peaks.append(freq)

    def get_loss(self, geo, context=None):

        # evolution_nr = get_app().get_service(MultiEvolution).evolution_nr

        evolution_nr = 1
        if evolution_nr == 1:
            f = np.arange(1, 1000, 2)
            segments = create_segments(geo)
            i = compute_impedance(segments, f)
        else:
            f, i = compute_impedance_iteratively(geo, n_precision_peaks=5)
        notes = get_notes(f,i)

        fundamental=single_note_loss(-31, notes)*self.weights["fundamental_loss"]
        octave=single_note_loss(-19, notes, i_note=1)*self.weights["octave_loss"]

        #notes=geo.get_cadsd().get_notes()
        tuning_loss=0
        volume_loss=0

        start_index=1
        if self.add_octave:
            start_index+=1
        if len(notes)>start_index:
            for ix, note in notes[start_index:].iterrows():
                f1=math.log(note["freq"],2)
                closest_target_index=np.argmin([abs(x-f1) for x in self.target_peaks])
                f2=self.target_peaks[closest_target_index]
                tuning_loss += math.sqrt(abs(f1-f2))
                volume_loss += math.sqrt(1/(note["impedance"]/1e6))

        tuning_loss*=self.weights["tuning_loss"]
        volume_loss*=self.weights["volume_loss"]
        
        n_notes=self.n_notes+1
        if self.add_octave:
            n_notes+=1
        n_note_loss=max(n_notes-len(notes), 0)*self.weights["n_note_loss"]

        d_loss = diameter_loss(geo)*self.weights["diameter_loss"]

        loss={
            "tuning_loss": tuning_loss,
            "volume_loss": volume_loss,
            "n_note_loss": n_note_loss,
            "diameter_loss": d_loss,
            "fundamental_loss": fundamental,
            "octave_loss": octave,
        }
        loss["loss"]=sum(loss.values())
        return loss

loss=MbeyaLoss(n_notes=5, add_octave=True)    
geo = [[0,32], [1500, 70]]
geo = Geo(geo)
loss.get_loss(geo)

  note_name  cent_diff  note_nr  freq  impedance   rel_imp
0       C#1   7.402308      -32    69   9.505698  1.000000
1        E2  19.157871      -17   163   4.336953  0.456248
2        C3 -22.186603       -9   265   2.483958  0.261313
3       F#3  -4.698796       -3   371   1.897939  0.199663
4       A#4 -39.782891        1   477   1.745377  0.183614
5        D4   6.880279        5   585   1.777744  0.187019
6        F4   3.613841        8   697   1.919174  0.201897
7       G#4  49.921820       11   807   2.115487  0.222549
8       A#5  21.162841       13   921   2.338299  0.245989
  note_name  cent_diff  note_nr  freq  impedance   rel_imp
0       C#1   7.402308      -32    69   9.505698  1.000000
1        E2  19.157871      -17   163   4.336953  0.456248
2        C3 -22.186603       -9   265   2.483958  0.261313
3       F#3  -4.698796       -3   371   1.897939  0.199663
4       A#4 -39.782891        1   477   1.745377  0.183614
5        D4   6.880279        5   585   1.777744  0.1870

{'tuning_loss': 9.863943844698317,
 'volume_loss': 2465.3186975383987,
 'n_note_loss': 0,
 'diameter_loss': 0.019000000000000003,
 'fundamental_loss': 2.3933497651705737,
 'octave_loss': 1.5528130552761392,
 'loss': 2479.1478042035437}